## Assignment 4
### Foundations of Machine Learning (CS564)

### *Feed Forward Neural Network for Document Classification of the BBC News Dataset*

<table style=\"font-size:25px\">
    <thead>
        <td><b>Name of Student</b></td>
        <td><b>Roll No.</b></td>
        <td><b>Date</b></td>
    </thead>
    <tr>
        <td>M. Maheeth Reddy</td>
        <td>1801CS31</td>
        <td>09-Nov-2021</td>
    </tr>
</table>

# Import Modules

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report

from tensorflow.keras.optimizers import Adam, SGD

import keras
from keras.callbacks import EarlyStopping
from keras.layers import Input, Embedding, Flatten, Dropout, Dense
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
import re

# Loading data

In [2]:
dataset = pd.read_csv("bbc.csv")
label_enc = LabelEncoder()
input_labels = label_enc.fit_transform(dataset['Class'])
label_enc_name_map = dict(zip(label_enc.classes_, label_enc.transform(label_enc.classes_)))
print(label_enc_name_map)
print(dataset.head())

{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
   Unnamed: 0                                            Article     Class
0           0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1           1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2           2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3           3  High fuel prices hit BA's profits\n\nBritish A...  business
4           4  Pernod takeover talk lifts Domecq\n\nShares in...  business


# Pre-processing data

In [3]:
porter_stemmer = PorterStemmer()
doc_list = dataset['Article'].tolist()
processed_list = []
for doc in doc_list:
    doc_str = re.sub(r'[^\w\s]', '', remove_stopwords(doc))
    doc_str_stem_lst = [porter_stemmer.stem(word) for word in doc_str.split(" ")]
    doc_str_stem = " ".join(doc_str_stem_lst)
    processed_list.append(doc_str_stem)

def get_tokenizer(data):
    print('Training tokenizer...')
    tokenizer = Tokenizer()
    print('Read {} Sentences'.format(len(data)))
    tokenizer.fit_on_texts(data)
    return tokenizer

def get_data(tokenizer, MAX_LENGTH, input_data, input_labels):
    print('Loading data')
    assert len(input_data) == len(input_labels)
    sequences = tokenizer.texts_to_sequences(input_data)
    X = pad_sequences(sequences, maxlen=MAX_LENGTH)
    Y_bcc = np.array(input_labels)
    return X, Y_bcc

tokenizer = get_tokenizer(processed_list)

X, Y_bcc = get_data(tokenizer, 50, processed_list, input_labels)
y_bcc_labels = keras.utils.np_utils.to_categorical(Y_bcc)
print(y_bcc_labels)

Training tokenizer...
Read 1912 Sentences
Loading data
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


# Split BBC News Dataset in 70:10:20 ratio

In [4]:
train1_X, test_X, train1_Y, test_Y = train_test_split(X, y_bcc_labels, test_size=0.2, random_state=43)
train_X, val_X, train_Y, val_Y = train_test_split(train1_X, train1_Y, test_size=0.1, random_state=43)

print('Getting Text FF NN')
input_layer = Input(shape=(50, ), dtype='int32') # No. of articles per batch is 50

Getting Text FF NN


# Hyper-parameters

In [5]:
max_word_count = 1+len(tokenizer.word_index)
embedding_layer = Embedding(max_word_count, 100, trainable=True) # Output Dimension for Embedding Layer = 100
embedded_input = embedding_layer(input_layer)
flatten_input = Flatten()(embedded_input)

# feed forward neural network
dense_layer_1 = Dense(2048, activation='relu')
drop_1 = Dropout(0.1)
dense_layer_2 = Dense(1024, activation='relu')
drop_2 = Dropout(0.1)
dense_layer_3 = Dense(512, activation='relu')
drop_3 = Dropout(0.1)
dense_layer_4 = Dense(5, activation='softmax')
output_layer = dense_layer_4(drop_3(dense_layer_3(drop_2(dense_layer_2(drop_1(dense_layer_1(flatten_input)))))))
model = Model(inputs=input_layer, outputs=output_layer)
print(model.summary())

# learning rate is 10^(-3), decay is 2*10^(-5)
lr = 1e-3
opt = SGD(learning_rate=lr)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           2253200   
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              10242048  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0     

# K-fold evaluation, overall accuracy

In [6]:
overall_acc = 0
split_num = 1
stop = [EarlyStopping(monitor='val_loss', patience=1)]
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(train_X): 
    history = model.fit(x=train_X[train_index], y=train_Y[train_index], batch_size=512, epochs=100, 
                        validation_data=(train_X[test_index], train_Y[test_index]), shuffle=True, callbacks=stop)
    predictions = model.predict(test_X)
    y_pred = [idx for idx in np.argmax(predictions, axis=1)]
    y_true = [idx for idx in np.argmax(test_Y, axis=1)]
    print('Accuracy on fold number {} is:'.format(split_num))
    split_num += 1
    acc_temp = metrics.accuracy_score(y_true, y_pred)*100
    print(acc_temp)
    overall_acc += acc_temp
    print(classification_report(y_true, y_pred, target_names = list(label_enc_name_map.keys())))
    
print("Overall Accuracy is: {}".format(overall_acc/3))

Epoch 1/100
2/2 [==============================] - 1s 490ms/step - loss: 1.6089 - accuracy: 0.2050 - val_loss: 1.6076 - val_accuracy: 0.2440
Epoch 2/100
2/2 [==============================] - 1s 360ms/step - loss: 1.6083 - accuracy: 0.2203 - val_loss: 1.6075 - val_accuracy: 0.2462
Epoch 3/100
2/2 [==============================] - 1s 346ms/step - loss: 1.6081 - accuracy: 0.2181 - val_loss: 1.6074 - val_accuracy: 0.2462
Epoch 4/100
2/2 [==============================] - 1s 357ms/step - loss: 1.6087 - accuracy: 0.2236 - val_loss: 1.6073 - val_accuracy: 0.2505
Epoch 5/100
2/2 [==============================] - 1s 293ms/step - loss: 1.6087 - accuracy: 0.2050 - val_loss: 1.6073 - val_accuracy: 0.2527
Epoch 6/100
2/2 [==============================] - 1s 309ms/step - loss: 1.6086 - accuracy: 0.2356 - val_loss: 1.6072 - val_accuracy: 0.2484
Epoch 7/100
2/2 [==============================] - 1s 300ms/step - loss: 1.6084 - accuracy: 0.2214 - val_loss: 1.6071 - val_accuracy: 0.2505
Epoch 8/100
2

Epoch 59/100
2/2 [==============================] - 1s 375ms/step - loss: 1.6053 - accuracy: 0.2421 - val_loss: 1.6029 - val_accuracy: 0.2854
Epoch 60/100
2/2 [==============================] - 1s 377ms/step - loss: 1.6056 - accuracy: 0.2334 - val_loss: 1.6029 - val_accuracy: 0.2898
Epoch 61/100
2/2 [==============================] - 1s 384ms/step - loss: 1.6047 - accuracy: 0.2639 - val_loss: 1.6028 - val_accuracy: 0.2898
Epoch 62/100
2/2 [==============================] - 1s 381ms/step - loss: 1.6048 - accuracy: 0.2552 - val_loss: 1.6027 - val_accuracy: 0.2876
Epoch 63/100
2/2 [==============================] - 1s 405ms/step - loss: 1.6048 - accuracy: 0.2541 - val_loss: 1.6026 - val_accuracy: 0.2876
Epoch 64/100
2/2 [==============================] - 1s 389ms/step - loss: 1.6049 - accuracy: 0.2508 - val_loss: 1.6026 - val_accuracy: 0.2941
Epoch 65/100
2/2 [==============================] - 1s 399ms/step - loss: 1.6046 - accuracy: 0.2410 - val_loss: 1.6025 - val_accuracy: 0.2963
Epoch 

/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

2/2 [==============================] - 1s 459ms/step - loss: 1.6008 - accuracy: 0.3021 - val_loss: 1.6026 - val_accuracy: 0.2614
Epoch 2/100
2/2 [==============================] - 1s 453ms/step - loss: 1.6007 - accuracy: 0.2803 - val_loss: 1.6026 - val_accuracy: 0.2614
Epoch 3/100
2/2 [==============================] - 1s 379ms/step - loss: 1.6004 - accuracy: 0.2770 - val_loss: 1.6025 - val_accuracy: 0.2636
Epoch 4/100
2/2 [==============================] - 1s 380ms/step - loss: 1.6011 - accuracy: 0.2770 - val_loss: 1.6024 - val_accuracy: 0.2614
Epoch 5/100
2/2 [==============================] - 1s 382ms/step - loss: 1.6008 - accuracy: 0.2781 - val_loss: 1.6024 - val_accuracy: 0.2614
Epoch 6/100
2/2 [==============================] - 1s 392ms/step - loss: 1.6007 - accuracy: 0.2923 - val_loss: 1.6023 - val_accuracy: 0.2593
Epoch 7/100
2/2 [==============================] - 1s 393ms/step - loss: 1.5997 - accuracy: 0.2846 - val_loss: 1.6023 - val_accuracy: 0.2614
Epoch 8/100
2/2 [========

Epoch 59/100
2/2 [==============================] - 1s 418ms/step - loss: 1.5972 - accuracy: 0.2846 - val_loss: 1.5992 - val_accuracy: 0.2614
Epoch 60/100
2/2 [==============================] - 1s 381ms/step - loss: 1.5966 - accuracy: 0.2912 - val_loss: 1.5991 - val_accuracy: 0.2614
Epoch 61/100
2/2 [==============================] - 1s 386ms/step - loss: 1.5962 - accuracy: 0.2890 - val_loss: 1.5990 - val_accuracy: 0.2614
Epoch 62/100
2/2 [==============================] - 1s 414ms/step - loss: 1.5971 - accuracy: 0.2923 - val_loss: 1.5990 - val_accuracy: 0.2614
Epoch 63/100
2/2 [==============================] - 1s 378ms/step - loss: 1.5958 - accuracy: 0.2999 - val_loss: 1.5989 - val_accuracy: 0.2614
Epoch 64/100
2/2 [==============================] - 1s 379ms/step - loss: 1.5963 - accuracy: 0.2912 - val_loss: 1.5989 - val_accuracy: 0.2614
Epoch 65/100
2/2 [==============================] - 1s 376ms/step - loss: 1.5965 - accuracy: 0.2835 - val_loss: 1.5988 - val_accuracy: 0.2614
Epoch 

/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

2/2 [==============================] - 1s 454ms/step - loss: 1.5941 - accuracy: 0.2832 - val_loss: 1.5963 - val_accuracy: 0.2773
Epoch 2/100
2/2 [==============================] - 1s 396ms/step - loss: 1.5942 - accuracy: 0.2876 - val_loss: 1.5963 - val_accuracy: 0.2795
Epoch 3/100
2/2 [==============================] - 1s 403ms/step - loss: 1.5937 - accuracy: 0.2854 - val_loss: 1.5962 - val_accuracy: 0.2795
Epoch 4/100
2/2 [==============================] - 1s 385ms/step - loss: 1.5931 - accuracy: 0.2843 - val_loss: 1.5962 - val_accuracy: 0.2795
Epoch 5/100
2/2 [==============================] - 1s 426ms/step - loss: 1.5939 - accuracy: 0.2843 - val_loss: 1.5961 - val_accuracy: 0.2795
Epoch 6/100
2/2 [==============================] - 1s 381ms/step - loss: 1.5939 - accuracy: 0.2843 - val_loss: 1.5961 - val_accuracy: 0.2795
Epoch 7/100
2/2 [==============================] - 1s 397ms/step - loss: 1.5939 - accuracy: 0.2865 - val_loss: 1.5960 - val_accuracy: 0.2795
Epoch 8/100
2/2 [========

Epoch 59/100
2/2 [==============================] - 1s 421ms/step - loss: 1.5898 - accuracy: 0.2843 - val_loss: 1.5936 - val_accuracy: 0.2795
Epoch 60/100
2/2 [==============================] - 1s 413ms/step - loss: 1.5905 - accuracy: 0.2832 - val_loss: 1.5936 - val_accuracy: 0.2795
Epoch 61/100
2/2 [==============================] - 1s 424ms/step - loss: 1.5901 - accuracy: 0.2832 - val_loss: 1.5935 - val_accuracy: 0.2795
Epoch 62/100
2/2 [==============================] - 1s 429ms/step - loss: 1.5900 - accuracy: 0.2832 - val_loss: 1.5935 - val_accuracy: 0.2795
Epoch 63/100
2/2 [==============================] - 1s 419ms/step - loss: 1.5901 - accuracy: 0.2832 - val_loss: 1.5935 - val_accuracy: 0.2795
Epoch 64/100
2/2 [==============================] - 1s 387ms/step - loss: 1.5898 - accuracy: 0.2832 - val_loss: 1.5934 - val_accuracy: 0.2795
Epoch 65/100
2/2 [==============================] - 1s 401ms/step - loss: 1.5895 - accuracy: 0.2832 - val_loss: 1.5934 - val_accuracy: 0.2795
Epoch 

/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/maheeth/anaconda3/envs/tf_im2txt/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta